In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
import numpy as np
import torch

/home/erciyes/miniconda3/envs/tap/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import latentplan.utils as utils
import latentplan.datasets as datasets

pybullet build time: May 20 2022 19:45:31


fail to lead iql


In [5]:
class Parser(utils.Parser):
    dataset: str = 'halfcheetah-medium-expert-v2'
    config: str = 'config.vqvae'

In [6]:
args = Parser().parse_args('train')

[ utils/setup ] Reading config: config.vqvae:halfcheetah_medium_expert_v2
[ utils/setup ] Not using overrides | config: config.vqvae | dataset: halfcheetah_medium_expert_v2
[ utils/setup ] Saved args to /home/erciyes/logs/halfcheetah-medium-expert-v2/vae/vq/args.json


In [8]:
env_name = args.dataset if "-v" in args.dataset else args.dataset+"-v0"
env = datasets.load_environment(env_name)

sequence_length = args.subsampled_sequence_length * args.step
args.logbase = os.path.expanduser(args.logbase)
args.savepath = os.path.expanduser(args.savepath)
if not os.path.exists(args.savepath):
    os.makedirs(args.savepath)

In [23]:
dataset_class = datasets.SequenceDataset

dataset_config = utils.Config(
    dataset_class,
    savepath=(args.savepath, 'data_config.pkl'),
    env=args.dataset,
    penalty=args.termination_penalty,
    sequence_length=sequence_length,
    step=args.step,
    discount=args.discount,
    disable_goal=args.disable_goal,
    normalize_raw=args.normalize,
    normalize_reward=args.normalize_reward,
    max_path_length=int(args.max_path_length)
)


Config: <class 'latentplan.datasets.sequence.SequenceDataset'>
    disable_goal: False
    discount: 0.99
    env: halfcheetah-medium-expert-v2
    max_path_length: 1000
    normalize_raw: True
    normalize_reward: True
    penalty: -100
    sequence_length: 25
    step: 1

Saved config to: /home/erciyes/logs/halfcheetah-medium-expert-v2/vae/vq/data_config.pkl



In [24]:
# Get dataset
dataset = dataset_config()
obs_dim = dataset.observation_dim
act_dim = dataset.action_dim
if args.task_type == "locomotion":
    transition_dim = obs_dim+act_dim+3
else:
    transition_dim = 128+act_dim+3

[ datasets/sequence ] Sequence length: 25 | Step: 1 | Max path length: 1000
[ datasets/sequence ] Loading... Downloading dataset: http://rail.eecs.berkeley.edu/datasets/offline_rl/gym_mujoco_v2/halfcheetah_medium_expert-v2.hdf5 to /home/erciyes/.d4rl/datasets/halfcheetah_medium_expert-v2.hdf5


load datafile: 100%|██████████| 9/9 [00:04<00:00,  1.82it/s]


✓
[ datasets/sequence ] Segmenting... ✓


In [27]:
loader = torch.utils.data.dataloader.DataLoader(dataset, shuffle=True, pin_memory=True,
                    batch_size=args.batch_size,
                    num_workers=0)

In [28]:
BATCH = next(iter(loader))

In [1]:
args.dataset

NameError: name 'args' is not defined